In [153]:
import numpy as np
import sympy as sym
import json
import matplotlib.pyplot as plt
from scipy import linalg
from scipy.interpolate import interp1d
from IPython.display import display, IFrame, HTML

# Controller Equations

\begin{gather}
\dot{x} = Ax + Bu \\
A = \frac{\partial f}{\partial x} = \frac{\partial f}{\partial s} \quad\quad B = \frac{\partial f}{\partial u} = \frac{\partial f}{\partial i} \\
u = -Kx \\
x = s - s_{eq} \quad\quad u = i - i_{eq} \\
s = [o_x, o_y, o_z, \psi, \theta, \phi, v_x, v_y, v_z, w_x, w_y, w_z] \\
u = [\tau_x, \tau_y, \tau_z, f_z]
\end{gather}

In [154]:
o_x, o_y, o_z, psi, theta, phi, v_x, v_y, v_z, w_x, w_y, w_z, a_z = sym.S("o_x, o_y, o_z, psi, theta, phi, v_x, v_y, v_z, w_x, w_y, w_z, a_z")
tau_x, tau_y, tau_z, f_z = sym.S("tau_x, tau_y, tau_z, f_z")

\begin{gather}
s_{eq} = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] \\
i_{eq} = [0, 0, 0, g]
\end{gather}

In [155]:
s = [o_x, o_y, o_z, psi, theta, phi, v_x, v_y, v_z]
i = [w_x, w_y, w_z, a_z]

In [156]:
x = []
u = []

# 2. Derive models

## 2.1 Define symbolic variables

Define states.

In [157]:
# components of position (meters)
o_x, o_y, o_z = sym.symbols('o_x, o_y, o_z')

# yaw, pitch, and roll angles (radians)
psi, theta, phi = sym.symbols('psi, theta, phi')

# components of linear velocity (meters / second)
v_x, v_y, v_z = sym.symbols('v_x, v_y, v_z')

Define inputs.

In [158]:
# gyroscope measurements - components of angular velocity (radians / second)
w_x, w_y, w_z = sym.symbols('w_x, w_y, w_z')

# z-axis accelerometer measurement - specific force (meters / second^2)
a_z = sym.symbols('a_z')

Define outputs.

In [159]:
n_x, n_y, r = sym.symbols('n_x, n_y, r')
a_0, a_1, a_2, a_3, a_4, a_5, a_6, a_7 = sym.symbols('a_0, a_1, a_2, a_3, a_4, a_5, a_6, a_7 ')

Define parameters.

In [160]:
g, k_flow = sym.symbols('g, k_flow')

Create linear and angular velocity vectors (in coordinates of the body frame).

In [161]:
v_01in1 = sym.Matrix([[v_x], [v_y], [v_z]])
w_01in1 = sym.Matrix([[w_x], [w_y], [w_z]])

## 2.2 Define kinematics of orientation

### 2.2.1 Rotation matrix in terms of yaw, pitch, roll angles

Define individual rotation matrices.

In [162]:
Rz = sym.Matrix([[sym.cos(psi), -sym.sin(psi), 0],
                 [sym.sin(psi), sym.cos(psi), 0],
                 [0, 0, 1]])

Ry = sym.Matrix([[sym.cos(theta), 0, sym.sin(theta)],
                 [0, 1, 0],
                 [-sym.sin(theta), 0, sym.cos(theta)]])

Rx = sym.Matrix([[1, 0, 0],
                 [0, sym.cos(phi), -sym.sin(phi)],
                 [0, sym.sin(phi), sym.cos(phi)]])

Apply sequential transformation to compute the rotation matrix that describes the orientation of the drone (i.e., of frame 1 in the coordinates of frame 0).

In [163]:
R_1in0 = Rz * Ry * Rx

### 2.2.2 Map from angular velocity to angular rates

Recall that

$$\begin{bmatrix} \dot{\psi} \\ \dot{\theta} \\ \dot{\phi} \end{bmatrix} = N w_{0, 1}^{1}$$

for some matrix $N$. Here is how to compute that matrix for a ZYX (yaw, pitch, roll) Euler angle sequence.  First, we compute its inverse:

In [164]:
Ninv = sym.Matrix.hstack((Ry * Rx).T * sym.Matrix([[0], [0], [1]]),
                              (Rx).T * sym.Matrix([[0], [1], [0]]),
                                       sym.Matrix([[1], [0], [0]]))

Then, we compute $N$ by taking the inverse of $N^{-1}$:

In [165]:
N = sym.simplify(Ninv.inv())

## 2.3 Derive equations of motion

Ratio of net thrust to mass in terms of z-axis accelerometer measurement.

In [166]:
f_z_over_m = a_z + (w_01in1.cross(v_01in1))[2]

Ratio of forces to mass.

In [167]:
f_in1_over_m = R_1in0.T * sym.Matrix([[0], [0], [-g]]) + sym.Matrix([[0], [0], [f_z_over_m]])

Equations of motion.

In [168]:
f = sym.Matrix.vstack(
    R_1in0 * v_01in1,
    N * w_01in1,
    (f_in1_over_m - w_01in1.cross(v_01in1)),
)

Show equations of motion, which have the form

$$\dot{s} = f(s, i, p)$$

where

$$
s = \begin{bmatrix} o_x \\ o_y \\ o_z \\ \psi \\ \theta \\ \phi \\ v_x \\ v_y \\ v_z \end{bmatrix}
\qquad\qquad
i = \begin{bmatrix} w_x \\ w_y \\ w_z \\ a_z \end{bmatrix}
\qquad\qquad
p = \begin{bmatrix} g \\ k_\text{flow} \end{bmatrix}.
$$

In [169]:
f

Matrix([
[ v_x*cos(psi)*cos(theta) + v_y*(sin(phi)*sin(theta)*cos(psi) - sin(psi)*cos(phi)) + v_z*(sin(phi)*sin(psi) + sin(theta)*cos(phi)*cos(psi))],
[v_x*sin(psi)*cos(theta) + v_y*(sin(phi)*sin(psi)*sin(theta) + cos(phi)*cos(psi)) + v_z*(-sin(phi)*cos(psi) + sin(psi)*sin(theta)*cos(phi))],
[                                                                       -v_x*sin(theta) + v_y*sin(phi)*cos(theta) + v_z*cos(phi)*cos(theta)],
[                                                                                         w_y*sin(phi)/cos(theta) + w_z*cos(phi)/cos(theta)],
[                                                                                                               w_y*cos(phi) - w_z*sin(phi)],
[                                                                                   w_x + w_y*sin(phi)*tan(theta) + w_z*cos(phi)*tan(theta)],
[                                                                                                          g*sin(theta) + v_y*w_z - v_z*w_y

# Observer Equations

In [170]:
o_x, o_y, o_z = sym.S("o_x, o_y, o_z") # drone position in abosulte frame -> loco pos frame
x_0, y_0, z_0 = sym.S("x_0, y_0, z_0")
x_1, y_1, z_1 = sym.S("x_1, y_1, z_1")
x_2, y_2, z_2 = sym.S("x_2, y_2, z_2")
x_3, y_3, z_3 = sym.S("x_3, y_3, z_3")
x_4, y_4, z_4 = sym.S("x_4, y_4, z_4")
x_5, y_5, z_5 = sym.S("x_5, y_5, z_5")
x_6, y_6, z_6 = sym.S("x_6, y_6, z_6")
x_7, y_7, z_7 = sym.S("x_7, y_7, z_7")

In [171]:
psi, theta, phi = sym.S("psi, theta, phi")
v_x, v_y, v_z = sym.S("v_x, v_y, v_z")

In [172]:
w_x, w_y, w_z = sym.S("w_x, w_y, w_z")
a_z = sym.S("a_z")

In [173]:
n_x, n_y, r = sym.S("n_x, n_y, r")

In [174]:
g, k_flow = sym.S("g, k_flow")

In [175]:
anchor_pos_0 = sym.sqrt((o_x-x_0)**2 + (o_y-y_0)**2 + (o_z-z_0)**2)
anchor_pos_1 = sym.sqrt((o_x-x_1)**2 + (o_y-y_1)**2 + (o_z-z_1)**2)
anchor_pos_2 = sym.sqrt((o_x-x_2)**2 + (o_y-y_2)**2 + (o_z-z_2)**2)
anchor_pos_3 = sym.sqrt((o_x-x_3)**2 + (o_y-y_3)**2 + (o_z-z_3)**2)
anchor_pos_4 = sym.sqrt((o_x-x_4)**2 + (o_y-y_4)**2 + (o_z-z_4)**2)
anchor_pos_5 = sym.sqrt((o_x-x_5)**2 + (o_y-y_5)**2 + (o_z-z_5)**2)
anchor_pos_6 = sym.sqrt((o_x-x_6)**2 + (o_y-y_6)**2 + (o_z-z_6)**2)
anchor_pos_7 = sym.sqrt((o_x-x_7)**2 + (o_y-y_7)**2 + (o_z-z_7)**2)

In [176]:
h = sym.Matrix([
    anchor_pos_0,
    anchor_pos_1,
    anchor_pos_2,
    anchor_pos_3,
    anchor_pos_4,
    anchor_pos_5,
    anchor_pos_6,
    anchor_pos_7,
    k_flow*(v_x-o_z*w_y)/o_z, # n_x
    k_flow*(v_y+o_z*w_x)/o_z, # n_y
    o_z / (sym.cos(phi)*sym.cos(theta)) # r
])
h

Matrix([
[sqrt((o_x - x_0)**2 + (o_y - y_0)**2 + (o_z - z_0)**2)],
[sqrt((o_x - x_1)**2 + (o_y - y_1)**2 + (o_z - z_1)**2)],
[sqrt((o_x - x_2)**2 + (o_y - y_2)**2 + (o_z - z_2)**2)],
[sqrt((o_x - x_3)**2 + (o_y - y_3)**2 + (o_z - z_3)**2)],
[sqrt((o_x - x_4)**2 + (o_y - y_4)**2 + (o_z - z_4)**2)],
[sqrt((o_x - x_5)**2 + (o_y - y_5)**2 + (o_z - z_5)**2)],
[sqrt((o_x - x_6)**2 + (o_y - y_6)**2 + (o_z - z_6)**2)],
[sqrt((o_x - x_7)**2 + (o_y - y_7)**2 + (o_z - z_7)**2)],
[                           k_flow*(-o_z*w_y + v_x)/o_z],
[                            k_flow*(o_z*w_x + v_y)/o_z],
[                             o_z/(cos(phi)*cos(theta))]])

In [177]:
s = [o_x, o_y, o_z, psi, theta, phi, v_x, v_y, v_z]
i = [w_x, w_y, w_z, a_z]
o = [anchor_pos_0,
    anchor_pos_1,
    anchor_pos_2,
    anchor_pos_3,
    anchor_pos_4,
    anchor_pos_5,
    anchor_pos_6,
    anchor_pos_7,
    n_x,
    n_y,
    r,
    ]
p = [g, k_flow]


In [178]:
o_x_eq = sym.symbols('o_x_eq')
o_y_eq = sym.symbols('o_y_eq')
o_z_eq = sym.symbols('o_z_eq')
s_eq = [o_x_eq, o_y_eq, o_z_eq, 0, 0, 0, 0, 0, 0]
i_eq = [0, 0, 0, g]

s_eq = [sym.nsimplify(a) for a in s_eq]
i_eq = [sym.nsimplify(a) for a in i_eq]


In [179]:
x = sym.Matrix([o_x, o_y, o_z, psi, theta, phi, v_x, v_y, v_z]) - sym.Matrix(s_eq)
u = sym.Matrix([w_x, w_y, w_z, a_z]) - sym.Matrix(i_eq)
y = sym.Matrix(o) - h.subs(tuple(zip(s, s_eq))).subs(tuple(zip(i, i_eq)))
y

Matrix([
[sqrt((o_x - x_0)**2 + (o_y - y_0)**2 + (o_z - z_0)**2) - sqrt((o_x_eq - x_0)**2 + (o_y_eq - y_0)**2 + (o_z_eq - z_0)**2)],
[sqrt((o_x - x_1)**2 + (o_y - y_1)**2 + (o_z - z_1)**2) - sqrt((o_x_eq - x_1)**2 + (o_y_eq - y_1)**2 + (o_z_eq - z_1)**2)],
[sqrt((o_x - x_2)**2 + (o_y - y_2)**2 + (o_z - z_2)**2) - sqrt((o_x_eq - x_2)**2 + (o_y_eq - y_2)**2 + (o_z_eq - z_2)**2)],
[sqrt((o_x - x_3)**2 + (o_y - y_3)**2 + (o_z - z_3)**2) - sqrt((o_x_eq - x_3)**2 + (o_y_eq - y_3)**2 + (o_z_eq - z_3)**2)],
[sqrt((o_x - x_4)**2 + (o_y - y_4)**2 + (o_z - z_4)**2) - sqrt((o_x_eq - x_4)**2 + (o_y_eq - y_4)**2 + (o_z_eq - z_4)**2)],
[sqrt((o_x - x_5)**2 + (o_y - y_5)**2 + (o_z - z_5)**2) - sqrt((o_x_eq - x_5)**2 + (o_y_eq - y_5)**2 + (o_z_eq - z_5)**2)],
[sqrt((o_x - x_6)**2 + (o_y - y_6)**2 + (o_z - z_6)**2) - sqrt((o_x_eq - x_6)**2 + (o_y_eq - y_6)**2 + (o_z_eq - z_6)**2)],
[sqrt((o_x - x_7)**2 + (o_y - y_7)**2 + (o_z - z_7)**2) - sqrt((o_x_eq - x_7)**2 + (o_y_eq - y_7)**2 + (o_z_eq - z_7)**2)],

In [180]:
A = f.jacobian(s).subs(tuple(zip(s, s_eq))).subs(tuple(zip(i, i_eq)))
B = f.jacobian(i).subs(tuple(zip(s, s_eq))).subs(tuple(zip(i, i_eq)))
A, B

(Matrix([
 [0, 0, 0, 0, 0,  0, 1, 0, 0],
 [0, 0, 0, 0, 0,  0, 0, 1, 0],
 [0, 0, 0, 0, 0,  0, 0, 0, 1],
 [0, 0, 0, 0, 0,  0, 0, 0, 0],
 [0, 0, 0, 0, 0,  0, 0, 0, 0],
 [0, 0, 0, 0, 0,  0, 0, 0, 0],
 [0, 0, 0, 0, g,  0, 0, 0, 0],
 [0, 0, 0, 0, 0, -g, 0, 0, 0],
 [0, 0, 0, 0, 0,  0, 0, 0, 0]]),
 Matrix([
 [0, 0, 0, 0],
 [0, 0, 0, 0],
 [0, 0, 0, 0],
 [0, 0, 1, 0],
 [0, 1, 0, 0],
 [1, 0, 0, 0],
 [0, 0, 0, 0],
 [0, 0, 0, 0],
 [0, 0, 0, 1]]))

In [181]:

C = h.jacobian(s).subs(tuple(zip(s, s_eq))).subs(tuple(zip(i, i_eq)))
D = h.jacobian(i).subs(tuple(zip(s, s_eq))).subs(tuple(zip(i, i_eq)))



In [182]:
s_obs_index = [0, 1, 2, 4, 5, 6, 7, 8]
A_obs = A[s_obs_index, :][:, s_obs_index]
B_obs = B[s_obs_index, :]
C_obs = C[:, s_obs_index]
D_obs = D

In [183]:
x_obs = sym.Matrix([o_x, o_y, o_z - o_z_eq, theta, phi, v_x, v_y, v_z])
C_obs*x_obs + D_obs*u - y

Matrix([
[o_x*(o_x_eq - x_0)/sqrt((o_x_eq - x_0)**2 + (o_y_eq - y_0)**2 + (o_z_eq - z_0)**2) + o_y*(o_y_eq - y_0)/sqrt((o_x_eq - x_0)**2 + (o_y_eq - y_0)**2 + (o_z_eq - z_0)**2) + (o_z - o_z_eq)*(o_z_eq - z_0)/sqrt((o_x_eq - x_0)**2 + (o_y_eq - y_0)**2 + (o_z_eq - z_0)**2) - sqrt((o_x - x_0)**2 + (o_y - y_0)**2 + (o_z - z_0)**2) + sqrt((o_x_eq - x_0)**2 + (o_y_eq - y_0)**2 + (o_z_eq - z_0)**2)],
[o_x*(o_x_eq - x_1)/sqrt((o_x_eq - x_1)**2 + (o_y_eq - y_1)**2 + (o_z_eq - z_1)**2) + o_y*(o_y_eq - y_1)/sqrt((o_x_eq - x_1)**2 + (o_y_eq - y_1)**2 + (o_z_eq - z_1)**2) + (o_z - o_z_eq)*(o_z_eq - z_1)/sqrt((o_x_eq - x_1)**2 + (o_y_eq - y_1)**2 + (o_z_eq - z_1)**2) - sqrt((o_x - x_1)**2 + (o_y - y_1)**2 + (o_z - z_1)**2) + sqrt((o_x_eq - x_1)**2 + (o_y_eq - y_1)**2 + (o_z_eq - z_1)**2)],
[o_x*(o_x_eq - x_2)/sqrt((o_x_eq - x_2)**2 + (o_y_eq - y_2)**2 + (o_z_eq - z_2)**2) + o_y*(o_y_eq - y_2)/sqrt((o_x_eq - x_2)**2 + (o_y_eq - y_2)**2 + (o_z_eq - z_2)**2) + (o_z - o_z_eq)*(o_z_eq - z_2)/sqrt((o_x_

In [184]:
anchors_x_sym = [x_0, x_1, x_2, x_3, x_4, x_5, x_6, x_7]
anchors_y_sym = [y_0, y_1, y_2, y_3, y_4, y_5, y_6, y_7]
anchors_z_sym = [z_0, z_1, z_2, z_3, z_4, z_5, z_6, z_7]
anchors_x = [-1.82, -3.57, 1.82, 2.67, -3.61, 1.84, 2.74, -1.85]
anchors_y = [2.18, 3.18, 2.21, -3.30, -3.30, -0.90, 3.18, -0.93]
anchors_z = [1.10, 2.24, 1.10, 2.20, 2.25, 1.10, 2.24, 1.09]

In [185]:
C_obs = C_obs.subs(tuple(zip(anchors_x_sym, anchors_x))).subs(tuple(zip(anchors_y_sym, anchors_y))).subs(tuple(zip(anchors_z_sym, anchors_z)))
C_obs = C_obs.subs(o_x_eq, 0).subs(o_y_eq, 0).subs(o_z_eq, 0).subs(k_flow, 0.01 * 30.0 / np.deg2rad(4.2))
C_obs


Matrix([
[ 0.597611727293697, -0.715820640384758, -0.361193901111575, 0, 0,   0,   0, 0],
[ 0.676178810479098, -0.602310537065415, -0.424269057555513, 0, 0,   0,   0, 0],
[-0.593413412403628, -0.720573429347262, -0.358656458046149, 0, 0,   0,   0, 0],
[-0.558449102736631,  0.690217992146398, -0.460145328097599, 0, 0,   0,   0, 0],
[  0.67053823392382,  0.612957388351414, -0.417925492057782, 0, 0,   0,   0, 0],
[-0.791399679501895,  0.387097669321579, -0.473119373615263, 0, 0,   0,   0, 0],
[-0.575884476562768, -0.668362275718833, -0.470796068430876, 0, 0,   0,   0, 0],
[ 0.790605509981734,  0.397439526639467, -0.465816219394644, 0, 0,   0,   0, 0],
[                 0,                  0,                  0, 0, 0, zoo,   0, 0],
[                 0,                  0,                  0, 0, 0,   0, zoo, 0],
[                 0,                  0,                  1, 0, 0,   0,   0, 0]])

In [186]:
A_arr = np.array(A_obs.subs(g, -9.81).tolist(), dtype=np.float64)
B_arr = np.array(B_obs.subs(g, -9.81).tolist(), dtype=np.float64)
C_arr = np.array(C_obs.tolist(), dtype=np.float64)

TypeError: Cannot convert complex to float

In [ ]:
def lqr(A, B, Q, R):
    P = linalg.solve_continuous_are(A, B, Q, R)
    K = linalg.inv(R) @  B.T @ P
    return K
Q = np.diag([
    1/0.049**2,
    1/0.072**2,
    1/0.046**2,
    1/0.114**2,
    1/0.063**2,
    1/0.048**2,
    1,
    1/0.038**2,
    1/1.696**2,             # n_x
    1/1.738**2,             # n_y
    1/0.148**2,             # r
])

R = np.diag([
    1/0.094**2,
    1/0.111**2,
    1/0.009**2, # o_z
    1/0.036**2, # theta
    1/0.030**2, # phi
    1/0.176**2, # v_x
    1/0.178**2, # v_y
    1/0.354**2, # v_z
])
L_1 = lqr(np.array(A_arr.tolist()).T, np.array(C_arr.tolist()).T, linalg.inv(R), linalg.inv(Q)).T
print(L_1.tolist())

[[0.321942986156017, 0.36009119735716655, -0.31836519002661123, -0.29990385211427484, 0.3556658073434917, -0.44223800573849864, -0.30859752381690714, 0.4400538559945905, 0.07155837375376561, -5.1254498950770905e-05, 0.006938422403778892], [-0.4215914013979726, -0.35088594367138437, -0.42313034301322366, 0.40542294400302886, 0.3559518086598309, 0.2373185196561766, -0.3918082691710136, 0.24175981326638982, -5.1786168251252376e-05, 0.07827675534935818, -0.026904856383195223], [-0.009786506620198865, -0.016606286688426532, -0.009893974509778313, -0.019082584534932362, -0.01709450885756774, -0.014419699419705488, -0.01880453992514826, -0.013810647633778608, 1.1114655588243871e-05, -4.071756593512843e-05, 2.2801892426333303], [-0.0007436490508847144, -0.0008320822148891163, 0.0007372765087559106, 0.0006925656903684675, -0.0008235263373806335, 0.0010221829030206179, 0.0007145652891975336, -0.0010184135151459872, -0.016451006130589963, 1.218280739697019e-07, -4.215242451832951e-06], [-0.001110

In [ ]:
L_str = np.array2string(L_1,
                        formatter={'float_kind': lambda x: f'{x:12.4f}'},
                        prefix='    ',
                        max_line_width=np.inf)

print(f'L = {L_str}')

L = [[      0.3219       0.3601      -0.3184      -0.2999       0.3557      -0.4422      -0.3086       0.4401       0.0716      -0.0001       0.0069]
     [     -0.4216      -0.3509      -0.4231       0.4054       0.3560       0.2373      -0.3918       0.2418      -0.0001       0.0783      -0.0269]
     [     -0.0098      -0.0166      -0.0099      -0.0191      -0.0171      -0.0144      -0.0188      -0.0138       0.0000      -0.0000       2.2802]
     [     -0.0007      -0.0008       0.0007       0.0007      -0.0008       0.0010       0.0007      -0.0010      -0.0165       0.0000      -0.0000]
     [     -0.0011      -0.0009      -0.0011       0.0011       0.0009       0.0006      -0.0010       0.0006      -0.0000       0.0207      -0.0000]
     [      0.0158       0.0177      -0.0156      -0.0147       0.0175      -0.0217      -0.0152       0.0216       0.2691      -0.0000       0.0002]
     [     -0.0217      -0.0181      -0.0218       0.0209       0.0183       0.0122      -0.0202    

In [ ]:
A_sym = sym.Matrix(A)
B_sym = sym.Matrix(B)
A_obs_sym = sym.Matrix(A_obs)
B_obs_sym = sym.Matrix(B_obs)
C_obs_sym = sym.Matrix(C_obs)
D_obs_sym = sym.Matrix(D_obs)
# L_0, L_1, L_2, L_3, L_4, L_5 = sym.symbols('L_0, L_1, L_2, L_3, L_4, L_5')
# L_sym = sym.Matrix([[0, 0, L_0],
#                     [L_1, 0, 0],
#                     [0, L_2, 0],
#                     [L_3, 0, 0],
#                     [0, L_4, 0],
#                     [0, 0, L_5]])
L_sym = sym.Matrix(L_1)
o_x, o_y, o_z = sym.symbols('o_x, o_y, o_z')
psi, theta, phi = sym.symbols('psi, theta, phi')
v_x, v_y, v_z = sym.symbols('v_x, v_y, v_z')
w_x, w_y, w_z = sym.symbols('w_x, w_y, w_z')
a_z, g, n_x, n_y, r = sym.symbols('a_z, g, n_x, n_y, r')
x_sym = sym.Matrix([o_x, o_y, o_z, psi, theta, phi, v_x, v_y, v_z])
x_obs_sym = sym.Matrix([o_x, o_y, o_z, theta, phi, v_x, v_y, v_z])
u_sym = sym.Matrix([w_x, w_y, w_z, a_z - g])
y_sym = sym.Matrix([a_0, a_1, a_2, a_3, a_4, a_5, a_6, a_7, n_x, n_y, r])
x_dot_sym = A_sym * x_sym + B_sym * u_sym
x_obs_dot_sym = A_obs_sym * x_obs_sym + B_obs_sym * u_sym
y_err_sym = C_obs_sym * x_obs_sym + D_obs_sym * u_sym - y_sym
x_hat_dot_sym = x_obs_dot_sym - L_sym * y_err_sym

In [187]:
Y_0, Y_1, Y_2, Y_3, Y_4, Y_5, Y_6, Y_7, Y_8, Y_9, Y_10 = sym.symbols('Y_0, Y_1, Y_2, Y_3, Y_4, Y_5, Y_6, Y_7, Y_8, Y_9, Y_10')
Y_err = sym.Matrix([Y_0, Y_1, Y_2, Y_3, Y_4, Y_5, Y_6, Y_7, Y_8, Y_9, Y_10])
(x_obs_dot_sym - L_sym * Y_err).tolist()

[[-0.321942986156017*Y_0 - 0.360091197357167*Y_1 - 0.00693842240377889*Y_10 + 0.318365190026611*Y_2 + 0.299903852114275*Y_3 - 0.355665807343492*Y_4 + 0.442238005738499*Y_5 + 0.308597523816907*Y_6 - 0.440053855994591*Y_7 - 0.0715583737537656*Y_8 + 5.12544989507709e-5*Y_9 + v_x],
 [0.421591401397973*Y_0 + 0.350885943671384*Y_1 + 0.0269048563831952*Y_10 + 0.423130343013224*Y_2 - 0.405422944003029*Y_3 - 0.355951808659831*Y_4 - 0.237318519656177*Y_5 + 0.391808269171014*Y_6 - 0.24175981326639*Y_7 + 5.17861682512524e-5*Y_8 - 0.0782767553493582*Y_9 + v_y],
 [0.00978650662019886*Y_0 + 0.0166062866884265*Y_1 - 2.28018924263333*Y_10 + 0.00989397450977831*Y_2 + 0.0190825845349324*Y_3 + 0.0170945088575677*Y_4 + 0.0144196994197055*Y_5 + 0.0188045399251483*Y_6 + 0.0138106476337786*Y_7 - 1.11146555882439e-5*Y_8 + 4.07175659351284e-5*Y_9 + v_z],
 [0.000743649050884714*Y_0 + 0.000832082214889116*Y_1 + 4.21524245183295e-6*Y_10 - 0.000737276508755911*Y_2 - 0.000692565690368467*Y_3 + 0.000823526337380634*Y

In [188]:
x_dot_sym

Matrix([
[    v_x],
[    v_y],
[    v_z],
[    w_z],
[    w_y],
[    w_x],
[g*theta],
[ -g*phi],
[a_z - g]])

In [189]:
x_hat_dot_sym

Matrix([
[                           0.321942986156017*a_0 + 0.360091197357167*a_1 - 0.318365190026611*a_2 - 0.299903852114275*a_3 + 0.355665807343492*a_4 - 0.442238005738499*a_5 - 0.308597523816907*a_6 + 0.440053855994591*a_7 + 5.12544989507709e-5*k_flow*w_x + 0.0715583737537656*k_flow*w_y + 0.0715583737537656*n_x - 5.12544989507709e-5*n_y - 2.02035526745161*o_x - 0.00273765932475445*o_y + 0.00388268786847761*o_z + 0.00693842240377889*r + 0.414286742155644*v_x + 0.000419523781562411*v_y],
[                              -0.421591401397973*a_0 - 0.350885943671384*a_1 - 0.423130343013224*a_2 + 0.405422944003029*a_3 + 0.355951808659831*a_4 + 0.237318519656177*a_5 - 0.391808269171014*a_6 + 0.24175981326639*a_7 - 0.0782767553493582*k_flow*w_x - 5.17861682512524e-5*k_flow*w_y - 5.17861682512524e-5*n_x + 0.0782767553493582*n_y - 0.00274524881569135*o_x - 1.85304908654786*o_y - 0.0134674081148235*o_z - 0.0269048563831952*r + 0.00042387555399302*v_x + 0.359296040643384*v_y],
[          -0.00978